The channel I watched is a Minecraft game one holded by #Tubbo 
https://www.twitch.tv/tubbo

The tutorial I used is:
https://www.learndatasci.com/tutorials/how-stream-text-data-twitch-sockets-python/

You may also need to enable twitch's 2-factor authentication.

This is a test round where I collected stream chat log from 10/18/2020 12:51 to 10/18/2020 12:58



In [ ]:
server = 'irc.chat.twitch.tv'
port = 6667
nickname = ''  # your username
token = '' # your token 
channel = '#tubbo'

In [ ]:
import socket
sock = socket.socket()

In [ ]:
sock.connect((server, port))

In [ ]:
sock.send(f"PASS {token}\n".encode('utf-8'))
sock.send(f"NICK {nickname}\n".encode('utf-8'))
sock.send(f"JOIN {channel}\n".encode('utf-8'))

12

In [ ]:
resp = sock.recv(2048).decode('utf-8')
resp

'd!5gahmed@5gahmed.tmi.twitch.tv PRIVMSG #tubbo :CHAAAAAAMPPPPP\r\n:moomoostake!moomoostake@moomoostake.tmi.twitch.tv PRIVMSG #tubbo :PogChampPogChampPogChampPogChamp\r\n:justbees_!justbees_@justbees_.tmi.twitch.tv PRIVMSG #tubbo :TUBBOOOO\r\n:stfleck!stfleck@stfleck.tmi.twitch.tv PRIVMSG #tubbo :TubbPog\r\n:rue_willings!rue_willings@rue_willings.tmi.twitch.tv PRIVMSG #tubbo :dreamw12DREAMWHEEZE dreamw12DREAMWHEEZE dreamw12DREAMWHEEZE dreamw12DREAMWHEEZE dreamw12DREAMWHEEZE dreamw12DREAMWHEEZE\r\n:me_gustaelpan!me_gustaelpan@me_gustaelpan.tmi.twitch.tv PRIVMSG #tubbo :ßßßßßßßßßßßßßßßßßßßßßßßßßßßßßßßßßßßßßßßßßßßßßßßß\r\n:ekn2497!ekn2497@ekn2497.tmi.twitch.tv PRIVMSG #tubbo :!uptime\r\n:luna_crystal_lycan1963!luna_crystal_lycan1963@luna_crystal_lycan1963.tmi.twitch.tv PRIVMSG #tubbo :get Tommy to go live!!!\r\n:streamelements!streamelements@streamelements.tmi.twitch.tv PRIVMSG #tubbo :@ekn2497, tubbo has been streaming for 1 hour 3 mins\r\n:fellowphanton!fellowphanton@fellowphanton.tmi.t

In [ ]:
import logging

logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s — %(message)s',
                    datefmt='%Y-%m-%d_%H:%M:%S',
                    handlers=[logging.FileHandler('chat.log', encoding='utf-8')])

In [ ]:
logging.info(resp)

In [ ]:
# Run this to collect data. Interrupt it to go to the next step

from emoji import demojize

while True:
    resp = sock.recv(2048).decode('utf-8')

    if resp.startswith('PING'):
        sock.send("PONG\n".encode('utf-8'))
    
    elif len(resp) > 0:
        logging.info(demojize(resp))

In [ ]:
# Check how is a single text message looks like
msg = '2020-10-18_12:58:35 — :gwnsnow!gwnsnow@gwnsnow.tmi.twitch.tv PRIVMSG #tubbo :why game german'

In [ ]:
from datetime import datetime

time_logged = msg.split()[0].strip()

time_logged = datetime.strptime(time_logged, '%Y-%m-%d_%H:%M:%S')

time_logged

datetime.datetime(2020, 10, 18, 12, 58, 35)

In [ ]:
username_message = msg.split('—')[1:]
username_message = '—'.join(username_message).strip()

username_message

':gwnsnow!gwnsnow@gwnsnow.tmi.twitch.tv PRIVMSG #tubbo :why game german'

In [ ]:
import re

username, channel, message = re.search(':(.*)\!.*@.*\.tmi\.twitch\.tv PRIVMSG #(.*) :(.*)', username_message).groups()

print(f"Channel: {channel} \nUsername: {username} \nMessage: {message}")

Channel: tubbo 
Username: gwnsnow 
Message: why game german


In [ ]:
import pandas as pd

def get_chat_dataframe(file):
    """
    Creates a dataframe with chat
    """
    data = []

    with open(file, 'r', encoding='utf-8') as f:
        lines = f.read().split('\n\n\n')
        
        for line in lines:
            try:
                time_logged = line.split('—')[0].strip()
                time_logged = datetime.strptime(time_logged, '%Y-%m-%d_%H:%M:%S')

                username_message = line.split('—')[1:]
                username_message = '—'.join(username_message).strip()

                username, channel, message = re.search(
                    ':(.*)\!.*@.*\.tmi\.twitch\.tv PRIVMSG #(.*) :(.*)', username_message
                ).groups()
                
                d = {
                    'dt': time_logged,
                    'channel': channel,
                    'username': username,
                    'message': message
                }

                data.append(d)
            
            except Exception:
                pass
            
    return pd.DataFrame().from_records(data)
        
    
df = get_chat_dataframe('chat.log')

In [ ]:
df.set_index('dt', inplace=True)

print(df.shape)

df

(2040, 3)


,channel,username,message
dt,,,
2020-10-18 12:51:06,tubbo,moomoostake,PogChampPogChampPogChampPogChamp
2020-10-18 12:51:09,tubbo,loopie123312,mamamama\
2020-10-18 12:51:09,tubbo,moomoostake,PogChamp PogChamp PogChamp PogChamp PogChamp P...
2020-10-18 12:51:09,tubbo,tommyinnitscool,hi
2020-10-18 12:51:10,tubbo,magedaworldstar,we don’t know tubbo? we know tubby
...,...,...,...
2020-10-18 12:58:37,tubbo,imthatstupid,POGG
2020-10-18 12:58:37,tubbo,bowelblaster1738,sootPopping_SG sootPopping_SG sootPopping_SG s...
2020-10-18 12:58:37,tubbo,remagamer7,NO TAKE IT


In [ ]:
sock.close()

In [ ]:
# Get only the message column as only that is required for the model prediction
df2 = df['message'].to_frame() 
df2 = df2.drop_duplicates() # drop duplicates

In [ ]:
game_platform = "minecraft_twitch"
df2.to_csv('../data/scraped/' + game_platform + '.csv', encoding='utf-8')

Next step: Could also see an audience's length of stay in the channel, find out the peaks and throughs of the number of audience, and see what triggers them to leave (whether that's associated with toxic behaviors). 